# Evaluacion Crediticia Preliminar con Perceptron
### Estructura de Datos y Algoritmos (T12A)

### Integrantes
* Castañeda De La Cruz, Edgar Daniel
* Jairo Villacorta

## Fases de Implementación

### F. Configuracion

Definicion de Hiperparametros y Carga de Datos en Memoria.

#### HIPERPARAMETROS

In [1]:
@eta = 0.00001 # TASA DE CONVERGENCIA (SUAVIZADO) 
@epochs = 10

10

#### CARGA DE DATOS A MEMORIA EN LISTA DE AMBITO GLOBAL @dataset

In [2]:
require 'csv'

@dataset = []

def cargar_data(file)
  # OBTENER CANTIDAD DE INPUTS
  i = 0
  
  CSV.foreach("#{file}.csv", :headers => true, converters: :numeric) do |row|
    @dataset[i] = row.fields
    i += 1
  end
  @ins = @dataset[i-1].size # CANTIDAD DE INPUTS
end

cargar_data('data')

8

#### GENERAR LISTA DE PESOS ALEATORIOS POR CADA INPUT (COLUMNA DE DATASET)

In [3]:
@w = []

i = 0
while i < @ins 
  @w[i] = rand(0..(@ins-1))/10.00
  i += 1
end

@w


[0.4, 0.0, 0.1, 0.4, 0.7, 0.0, 0.1, 0.4]

#### DEFINIR LA FUNCION DE ACTIVACION TIPO sigmoidal binaria 

In [4]:
def sigmoidal(inputs)
  i = 0
  weighted_average = 0
  inputs = [1] + inputs
  # OBTENER SUMA DEL PRODUCTO DE LOS PESOS
  while i < @ins
    wn = @w[i]
    xn = inputs[i]
    weighted_average += wn*xn
    i += 1
  end
  return weighted_average
end

def predict(inputs)
  weighted_average = sigmoidal(inputs)
  if weighted_average > 0
    return 1  
  else
    return 0
  end
end

:predict

#### DEFINIR FUNCION DE ENTRENAMIENTO

In [5]:
def train(inputs, ex_output)
  output = predict(inputs)
  inputs = [1] + inputs
  error = ex_output - output
  if error != 0
      # SI EXISTE EL ERROR REAJUSTAR LOS PESOS
      i = 0
      while i < @ins
        wn = @w[i]
        xn = inputs[i]
        @w[i] = wn + (@eta*error*xn)
        i += 1
      end
  end
  return error
end              

:train

### F. Entrenamiento

In [6]:
weights = [] # Lista con los pesos
errors = []  # Lista con los errores

data_x = []
data_y = []
i = 0
while i <= @epochs
  # ENTRENAREMOS VARIAS VECES SOBRE EL MISMO DATASET
  # PARA QUE LOS PESOS CONVERGAN
  sum_errors = 0
  @dataset.each_with_index do | person, index |
    # OBTENER EL ULTIMO ELEMENTO DE LA LISTA
    output = person[-1]
    inp = person.take person.size - 1
    weights.push(@w)
    err = train(inp, output)
    errors.push(err)
    sum_errors += err
  end
  # GUARDAR DATOS PARA LA REPRESENTACION
  data_y.push(@ins - sum_errors.round(2))
  data_x.push(i)
  i += 1
end


#### REPRESENTACION DE LA PRECICION EN CADA ITERACION

In [7]:
require 'nyaplot'

plot = Nyaplot::Plot.new
line_pres = plot.add(:line, data_x, data_y)
line_pres.color('#ff2680')

plot.x_label("Iteraciones")
plot.y_label("Errores")
plot.show

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

#<Nyaplot::Frame:0x00000001807f18 @properties={:panes=>[#<Nyaplot::Plot:0x00000001835ad0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x00000001834608 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :color=>"#ff2680"}, :data=>"5722bf9e-039c-4ef1-92b4-ce8799b4e78d"}, @xrange=[0, 10], @yrange=[20.0, 98.0]>], :options=>{:x_label=>"Iteraciones", :y_label=>"Errores", :zoom=>true, :width=>700, :xrange=>[0, 10], :yrange=>[20.0, 98.0]}}>], :data=>{"5722bf9e-039c-4ef1-92b4-ce8799b4e78d"=>#<Nyaplot::DataFrame:0x00000001835580 @name="5722bf9e-039c-4ef1-92b4-ce8799b4e78d", @rows=[{:data0=>0, :data1=>98.0}, {:data0=>1, :data1=>55.0}, {:data0=>2, :data1=>46.0}, {:data0=>3, :data1=>36.0}, {:data0=>4, :data1=>33.0}, {:data0=>5, :data1=>30.0}, {:data0=>6, :data1=>26.0}, {:data0=>7, :data1=>22.0}, {:data0=>8, :data1=>20.0}, {:data0=>9, :data1=>21.0}, {:data0=>10, :data1=>20.0}]>}, :extension=>[]}>

#### REPRESENTACION DE LA DISTRIBUCION DEL PATRON

In [8]:
data_x = [[], []]
data_y = [[], []]

@dataset.each_with_index do | person, index |
  # OBTENER EL ULTIMO ELEMENTO DE LA LISTA
  tag = person[-1]
  inp = person.take person.size - 1
  data_x[tag].push(sigmoidal(inp))
  data_y[tag].push(index)
end

plot = Nyaplot::Plot.new
line_pres = plot.add(:scatter, data_x[0], data_y[0])
line_pres.color('#ff2680')

line_pres = plot.add(:scatter, data_x[1], data_y[1])
line_pres.color('#15db29')

plot.show

#<Nyaplot::Frame:0x000000015394a8 @properties={:panes=>[#<Nyaplot::Plot:0x00000001688980 @properties={:diagrams=>[#<Nyaplot::Diagram:0x000000015e1f40 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"#ff2680"}, :data=>"972cc8f1-ce96-4442-8b5e-1eefc29a9875"}, @xrange=[-223.39671000000072, 18.630939999999804], @yrange=[3, 844]>, #<Nyaplot::Diagram:0x00000001539f48 @properties={:type=>:scatter, :options=>{:x=>"data0", :y=>"data1", :color=>"#15db29"}, :data=>"fb8cb3b5-dec0-4556-82aa-f8a7619aa1eb"}, @xrange=[-54.08861000000191, 766.5376099999991], @yrange=[0, 846]>], :options=>{:zoom=>true, :width=>700, :xrange=>[-223.39671000000072, 766.5376099999991], :yrange=>[0, 846]}}>], :data=>{"972cc8f1-ce96-4442-8b5e-1eefc29a9875"=>#<Nyaplot::DataFrame:0x0000000167bf00 @name="972cc8f1-ce96-4442-8b5e-1eefc29a9875", @rows=[{:data0=>-159.46398000000076, :data1=>3}, {:data0=>-83.11169000000035, :data1=>4}, {:data0=>-22.650950000000243, :data1=>9}, {:data0=>-130.08476000000056, :data1=>10}, {:data0=>-122.8853700000005, :data1=>13}, {:data0=>-154.26583000000062, :data1=>14}, {:data0=>-53.80302000000039, :data1=>15}, {:data0=>-14.628230000000242, :data1=>16}, {:data0=>-68.9032000000004, :data1=>18}, {:data0=>-39.40621000000031, :data1=>23}, {:data0=>-67.01170000000046, :data1=>24}, {:data0=>-178.8548700000007, :data1=>26}, {:data0=>-25.625400000000276, :data1=>27}, {:data0=>-176.66641000000075, :data1=>28}, {:data0=>-18.848960000000204, :data1=>29}, {:data0=>-92.86757000000038, :data1=>30}, {:data0=>-47.708730000000386, :data1=>31}, {:data0=>11.634649999999759, :data1=>33}, {:data0=>-131.88221000000064, :data1=>36}, {:data0=>-49.34007000000028, :data1=>37}, {:data0=>-154.8388300000007, :data1=>38}, {:data0=>-104.6234600000005, :data1=>39}, {:data0=>-176.8539100000007, :data1=>41}, {:data0=>-56.56363000000029, :data1=>42}, {:data0=>-26.24617000000022, :data1=>43}, {:data0=>-125.50179000000048, :data1=>46}, {:data0=>-102.98476000000036, :data1=>47}, {:data0=>-142.24208000000067, :data1=>49}, {:data0=>-33.11806000000029, :data1=>51}, {:data0=>-148.54068000000072, :data1=>52}, {:data0=>-115.13528000000058, :data1=>53}, {:data0=>-171.19191000000075, :data1=>54}, {:data0=>-86.04327000000042, :data1=>55}, {:data0=>-120.95007000000058, :data1=>59}, {:data0=>-110.70775000000052, :data1=>62}, {:data0=>-158.37035000000068, :data1=>68}, {:data0=>-114.89918000000044, :data1=>69}, {:data0=>-58.93156000000029, :data1=>73}, {:data0=>-100.11786000000053, :data1=>76}, {:data0=>-163.98494000000065, :data1=>77}, {:data0=>-142.48778000000055, :data1=>78}, {:data0=>-220.5760900000008, :data1=>80}, {:data0=>-37.37549000000027, :data1=>82}, {:data0=>-87.75291000000047, :data1=>83}, {:data0=>-66.44597000000039, :data1=>86}, {:data0=>-97.88847000000048, :data1=>91}, {:data0=>-125.43448000000053, :data1=>92}, {:data0=>-157.7789500000007, :data1=>95}, {:data0=>-125.17851000000067, :data1=>97}, {:data0=>-2.7008000000002834, :data1=>98}, {:data0=>-24.66526000000019, :data1=>101}, {:data0=>-151.9558600000006, :data1=>102}, {:data0=>-201.8223200000008, :data1=>104}, {:data0=>-187.3567300000007, :data1=>105}, {:data0=>-132.60582000000068, :data1=>106}, {:data0=>-148.82858000000067, :data1=>107}, {:data0=>-166.7625300000007, :data1=>108}, {:data0=>-140.06410000000065, :data1=>109}, {:data0=>-159.5796100000007, :data1=>115}, {:data0=>-16.631120000000244, :data1=>117}, {:data0=>-114.99936000000046, :data1=>118}, {:data0=>-101.56553000000052, :data1=>121}, {:data0=>-85.50437000000045, :data1=>122}, {:data0=>-66.93298000000036, :data1=>123}, {:data0=>-47.283060000000255, :data1=>124}, {:data0=>-112.40033000000051, :data1=>126}, {:data0=>-184.52379000000076, :data1=>127}, {:data0=>-126.90841000000046, :data1=>129}, {:data0=>-118.78864000000044, :data1=>130}, {:data0=>-113.71394000000052, :data1=>132}, {:data0=>-100.96442000000042, :data1=>134}, {:data0=>-99.64176000000049, :data1=>135}, {:data0=>-63.33966000000044, :data1=>137}, {:data0=>-166.96013000000067, :

## F. De Pruebas

### Cotización #1
Cliente: EMPRESA TURISTICA SOL Y MAR E.I.R.L. | *RUC: 20140917690*

#### Evaluación
Cliente percibe un ingreso fijo mensual de **S/. 2100.00**, ademas de justificar otros ingresos por facturas de servicio por **S/. 1600.00** Aprox, tiene con nosotros **15 Letras** Pagadas, el importe de su ultima letra cobrada fue de **S/. 110.00**, Edad del cliente al solicitar nuevo prestamo es de **36 años**. Pertenece al **Sector 7** (PYMES De Servicios), Antiguedad del cliente **7 meses**.


In [9]:
ingreso_mensual = 2100
otros_ingresos = 1600
facturas = 15
importe_ultima_fact = 110
edad = 36
meses_contrato = 7
sector = 7

resp = predict([
  ingreso_mensual,
  otros_ingresos,
  facturas,
  importe_ultima_fact,
  edad,
  meses_contrato,
  sector
])

if resp == 1
  print "Asignar Prestamo\n"
else
  print "NO Asignar Prestamo\n"
end


Asignar Prestamo


### Cotización #2
Cliente: SERVICIOS GENERALES DE ALQUILER A & J E.I.R.L. | *RUC: 20529360712*

#### Evaluación
Cliente percibe un ingreso fijo mensual de **S/. 1350.00**, ademas de justificar otros ingresos por facturas de servicio por **S/. 360.00** Aprox, tiene con nosotros **10 Letras** Pagadas, el importe de su ultima letra cobrada fue de **S/. 150.00**, Edad del cliente al solicitar nuevo prestamo es de **33 años**. Pertenece al **Sector 2** (PYMES Alquiler y Arrendamiento), Antiguedad del cliente **23 meses**.


In [10]:
ingreso_mensual = 1350
otros_ingresos = 360
facturas = 10
importe_ultima_fact = 150
edad = 33
meses_contrato = 23
sector = 2


resp = predict([
  ingreso_mensual,
  otros_ingresos,
  facturas,
  importe_ultima_fact,
  edad,
  meses_contrato,
  sector
])

if resp == 1
  print "Asignar Prestamo\n"
else
  print "NO Asignar Prestamo\n"
end

NO Asignar Prestamo
